<a href="https://colab.research.google.com/github/bes82/Deformable_Nondeformable/blob/main/Deformable_Nondeformable_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries, download network, and load network.

This cell has three jobs.

1.) Import useful libraries for use during the pipeline.

2.) Download the pipeline network architecture and weights from a Google Drive.

3.) Load the network for use in the pipeline.

In [1]:
#@title  <- Click here to run code. Double click this text to see code.

print("Importing Useful Libraries...")
# Importing Useful Libraries.
!pip install tensorflow==2.8.0
import tensorflow.keras
import tensorflow as tf
from tensorflow.keras.models import load_model, Model, model_from_json

import cv2, os
from scipy.ndimage import label
from skimage import measure
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import shutil
print("Done!")

# Clone Repository And Download Network Architecture and Weights.
print("Cloning Repository...")
! git clone https://github.com/bes82/Deformable_Nondeformable
print("Done!")
%cd Deformable_Nondeformable

ce_jaccard_Ids = ["162zLUhwsptTgKvQgcFP9gLr7k1c2Nycd", "1PVj5wu_1aBE6RFwAC42KJAVdPiBBdv6m",
"1zK8LevIF9OmqEMFUOsXy3Kq4EJHneJE0", "1y-kxB9OM7ApmERkTKk9WLSp20d5tVZjp",
"1bd83AiLVsDTOOzakoneKNuTF-0lErgp2", "1fOPC2X0IQwz1K41J_rnD7VyGgwbbuab9",
"14eXQj5z826ZAne53Oa2W_iH0LIWBy_rQ", "1LAWDBFL0XQ_v6HVhtbanTR5jnNssSZ0A",
"1NQjou3OFIkmvDn7hA9nnqDSZowk15qwd", "1ge7FtPuPhlMy6m57jM-xcoQuENSpFFeZ"]

resnet50_IDs = ["1zG8OurfrocofuPg6fFguI4CFIXicP1hE", "1fyH1FYqQ8_yAaZdqyYAcuQ2xsAWgJ63g",
"1Dteg7ZzF2NTIhguWb9l4pdf0nwgEIG2x", "1DB2Nl-BhioF_5nnaF6clf3-dP0JtQvO2",
"1-Y2zr5oYbhLyRAxxofipDh3Q61EOmAXr", "1qmZ3PnlmS0CkuT9JcO5WMyW7p8qB99xL",
"1I1csaKs1nfYLTzCyjBNxqQasCC1qVKB8", "17dBcJW4rcLNntNIb3MfaW8qp1H0hb9NH",
"10ar4xB2ip18FdoMwymosxbVRBO5sJ8zO", "16kTNv8l37nvAlyJi4J7zYvQcpnVBTEpA"]

#print(ce_jaccard_Ids)
print("Downloading Network Architecture And Weights...")

!gdown --id "162zLUhwsptTgKvQgcFP9gLr7k1c2Nycd"
!gdown --id "1PVj5wu_1aBE6RFwAC42KJAVdPiBBdv6m"
!gdown --id "1zK8LevIF9OmqEMFUOsXy3Kq4EJHneJE0"
!gdown --id "1y-kxB9OM7ApmERkTKk9WLSp20d5tVZjp"
!gdown --id "1bd83AiLVsDTOOzakoneKNuTF-0lErgp2"
!gdown --id "1fOPC2X0IQwz1K41J_rnD7VyGgwbbuab9"
!gdown --id "14eXQj5z826ZAne53Oa2W_iH0LIWBy_rQ"
!gdown --id "1LAWDBFL0XQ_v6HVhtbanTR5jnNssSZ0A"
!gdown --id "1NQjou3OFIkmvDn7hA9nnqDSZowk15qwd"
!gdown --id "1ge7FtPuPhlMy6m57jM-xcoQuENSpFFeZ"

!gdown --id "1zG8OurfrocofuPg6fFguI4CFIXicP1hE"
!gdown --id "1fyH1FYqQ8_yAaZdqyYAcuQ2xsAWgJ63g"
!gdown --id "1Dteg7ZzF2NTIhguWb9l4pdf0nwgEIG2x"
!gdown --id "1DB2Nl-BhioF_5nnaF6clf3-dP0JtQvO2"
!gdown --id "1-Y2zr5oYbhLyRAxxofipDh3Q61EOmAXr"
!gdown --id "1qmZ3PnlmS0CkuT9JcO5WMyW7p8qB99xL"
!gdown --id "1I1csaKs1nfYLTzCyjBNxqQasCC1qVKB8"
!gdown --id "17dBcJW4rcLNntNIb3MfaW8qp1H0hb9NH"
!gdown --id "10ar4xB2ip18FdoMwymosxbVRBO5sJ8zO"
!gdown --id "16kTNv8l37nvAlyJi4J7zYvQcpnVBTEpA"

network_Names = os.listdir()
network_Names.remove('README.md')
network_Names.remove('.git')

phase_1_Network_Names = []
phase_2_Network_Names = []
for name in network_Names:
  if 'Ensemble' in name:
    phase_1_Network_Names.append(name)
  elif 'Ensemble' not in name:
    phase_2_Network_Names.append(name)

phase_1_Ensemble = []
phase_2_Ensemble = []

for name in phase_1_Network_Names:
  if '.json' in name:
    json_file = open(name)
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(name[:-5] + ".h5")
    phase_1_Ensemble.append(loaded_model)

for name in phase_2_Network_Names:
  if '.json' in name:
    json_file = open(name)
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(name[:-5] + ".h5")
    phase_2_Ensemble.append(loaded_model)


print("Done!")

'''# Load Downloaded Network Architecture And Weights.
print("Loading Network Architecture and Weights...")
network_Name = "Motion_Blur_Network.h5"
network = tf.keras.models.load_model(network_Name)
print("Done!")'''

Importing Useful Libraries...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 42.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Fo

'# Load Downloaded Network Architecture And Weights.\nprint("Loading Network Architecture and Weights...")\nnetwork_Name = "Motion_Blur_Network.h5"\nnetwork = tf.keras.models.load_model(network_Name)\nprint("Done!")'

## Create directory to hold channels for analysis.

This cell is used to create a directory that will hold channels that we want to analyze.

IMPORTANT: Once this cell is run, you will need to manually upload channel images to the directory titled "Motion_Blur/Channel_Directory".

If you want to re-initialize the directory, then just re-run this code after analyzing your first batch of images.

In [2]:
#@title <- Click here to run code. Double click this text to view code.
# Create A Directory To Hold Channels For Analysis.
channel_Directory = "Channel_Directory/"
if os.path.isdir(channel_Directory):
  shutil.rmtree(channel_Directory)
os.mkdir(channel_Directory)

## Channel Analysis.

This cell applies analysis to all of the channels uploaded to the "Motion_Blur/Channel_Directory" directory. After each channel is analyzed, the sRBC counts are appended to a data frame and displayed to the user.

After all channels are analyzed, the final dataframe is displayed to the user.

The results are also saved as a .csv file under the name "Analysis_Output.csv" in the Deformable_Nondeformable/ directory on Colab.

In [4]:
#@title <- Click here to run code. Double click this text to view code.
# An array to hold pixel areas for all tested channels.

@tf.function
def phase_One_Predictions(ensemble,tile):
  for kfold, model in enumerate(ensemble):
    if kfold == 0:
      output = model(tile)
    else:
      output += model(tile)
  output *= 1/len(ensemble)
  return output

blob_Sizes = []
output_Data_Frame = pd.DataFrame(columns = ["File Name", "Adhered Objects", "Deformable Counts", "Non-Deformable Counts", "Other", "Time (s)"])
# A function which applies a zero mean normalization to the input images.
def standard_norm(img):
    height, width, channels = img.shape
    for channel in range(channels):
        img[:,:,channel] = (img[:,:,channel] - np.mean(img[:,:,channel]))/np.std(img[:,:,channel])
    return img

# Thresholds to be tested.
thresholds = [90]
padding = 20
# This large for loop will analyze all of the channels uploaded to the Channel Directory.
for image_Name in os.listdir(channel_Directory):
    # If statement necessary to avoid possible errors during analysis on Google Colab.
    if ".ipynb" in image_Name:
      continue
    # Start time of analysis.
    start_Time = time.time()
    print("Analyzing " + image_Name[:-4])
    # Reading In Channel.
    full_Channel = plt.imread(channel_Directory + image_Name)
    # Convert grayscale image to RGB if the input channel is grayscale.
    if len(np.shape(full_Channel)) == 2:
        full_Channel = cv2.cvtColor(full_Channel, cv2.COLOR_GRAY2RGB)
    # Defining characteristics of the input channel.
    image_Height, image_Width, channels = np.shape(full_Channel)
    # The following if statements are used to resize the channel to have
    # dimensions which are evenly dividable by 150. To avoid as much distortion
    # as possible, we will resize either up or down in each dimensions,
    # depending on which end the dimensions is closer to.
    if (image_Height % 150) < 75 and (image_Width % 150) < 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int(np.floor(image_Width/150)*150), int(np.floor(image_Height/150)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int(np.floor(image_Height/150))
        horizontal_Tiles = int(np.floor(image_Width/150))
    elif (image_Height % 150) >= 75 and (image_Width % 150) >= 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int((np.floor(image_Width/150) + 1)*150), int((np.floor(image_Height/150) + 1)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int((np.floor(image_Height/150) + 1))
        horizontal_Tiles = int((np.floor(image_Width/150) + 1))
    elif (image_Height % 150) >= 75 and (image_Width % 150) < 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int(np.floor(image_Width/150)*150), int((np.floor(image_Height/150) + 1)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int((np.floor(image_Height/150) + 1))
        horizontal_Tiles = int(np.floor(image_Width/150))
    else:
        full_Channel_Resized = cv2.resize(full_Channel,(int((np.floor(image_Width/150) + 1)*150), int(np.floor(image_Height/150)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int(np.floor(image_Height/150))
        horizontal_Tiles = int((np.floor(image_Width/150) + 1))
    full_Channel_Resized_Borders = cv2.copyMakeBorder(full_Channel_Resized.copy(), padding, padding, padding, padding, cv2.BORDER_CONSTANT)
    # Defining characteristics of the resized input channel.
    image_Height_Resized, image_Width_Resized, channels = np.shape(full_Channel_Resized)
    # Creating an array which will hold predictions.
    output_Image = np.zeros((image_Height_Resized,image_Width_Resized))

    # The following chunk of code will make predictions, and create output
    # images and mask predictions.
    x_Slider = 0
    y_Slider = 0
    # Creating an array which will hold predictions for each tile.
    output_Array = np.zeros((128,128))
    # In the following for loops, we will slide through the input channel,
    # tile by tile, and make predictions on each tile.
    for i in range(vertical_Tiles):
        x_Slider = 150*i
        # Sliding through all tiles in a row for each row.
        for j in range(horizontal_Tiles):
            y_Slider = 150*j
            # Resizing tile to required input size.
            current_Tile = full_Channel_Resized[x_Slider:x_Slider + 150, y_Slider: y_Slider + 150,:]/255
            current_Tile = cv2.resize(current_Tile, (128,128), interpolation=cv2.INTER_AREA)

            # Normalizing the tile.
            current_Tile_Normalized = standard_norm(current_Tile.copy())
            current_Tile_Normalized = current_Tile_Normalized[None,:,:,:]
            sample, height, width, depth = current_Tile_Normalized.shape
            empty_mask = np.zeros((sample, height, width, depth))
            output = phase_One_Predictions(phase_1_Ensemble,current_Tile_Normalized)
            #for kfold, model in enumerate(phase_1_Ensemble):
              #if kfold == 0:
                #output = model(current_Tile_Normalized)
              #else:
                #output += model(current_Tile_Normalized)
            #output *= 1/len(phase_1_Ensemble)

            # Finding the prediction for each pixel in the tile.
            #output = np.argmax(output, axis = 3)
            #output = np.reshape(output, (128,128,1))[:,:,0]
            output_Max = np.argmax(output, axis = 3)
            output_Array = output_Max[0,:,:].astype('float32')
            #for i in range(128):
                #for j in range(128):
                    #output_Array[i,j] = np.argmax(output[0,i,j,:])

            # Resizing tile back to original size.
            output_Array = cv2.resize(output_Array,(150,150),interpolation = cv2.INTER_AREA)
            output_Image[x_Slider:x_Slider + 150, y_Slider: y_Slider + 150] = output_Array
            output_Array = np.zeros((128,128))
    # The following for loops binarize the output, after resizing distortion.
    output_Image = np.around(output_Image).astype(int)
    #for i in range(image_Height_Resized):
        #for j in range(image_Width_Resized):
            #output_Image[i,j] = int(round(output_Image[i,j],1))

    # Defining connected pixel regions in the final image.
    blobs, number_Of_Blobs = label(output_Image == 1) # whole channel image
    properties = measure.regionprops(blobs)
    centroids = [prop.centroid for prop in properties if prop.area > 50]
    #print(len(centroids))
    deformable = 0
    nondeformable = 0
    other = 0
    for i in range(len(centroids)):
        cell_Region = full_Channel_Resized_Borders[int(centroids[i][0]) - 16 + padding:int(centroids[i][0]) + 16 + padding,int(centroids[i][1]) - 16 + padding:int(centroids[i][1]) + 16 + padding,:]
        cell_Region_Resized = cv2.resize(cell_Region.copy().astype('float64'), (224,224), interpolation=cv2.INTER_LINEAR)/255
        #cell_Count = cell_Count + 1
        cell_Region_Resized_Normalized = standard_norm(cell_Region_Resized.copy())
        cell_Region_Resized_Normalized = cell_Region_Resized_Normalized[None,:,:,:]
        for kfold, model in enumerate(phase_2_Ensemble):
            if kfold == 0:
                percentages = model(cell_Region_Resized_Normalized)
            else:
                percentages += model(cell_Region_Resized_Normalized)
        percentages *= 1/len(phase_2_Ensemble)
        prediction = np.argmax(percentages)
        if percentages[0][0] > 0.4:
            deformable = deformable + 1
        if percentages[0][1] > 0.4:
            nondeformable = nondeformable + 1
        if percentages[0][2] > 0.9:
            other = other + 1
    #print("Deformable =  " + str(deformable))
    #print("Nondeformable = " + str(nondeformable))
    #print("Other = " + str(other))
    end_Time = time.time()
    time_Change = end_Time - start_Time
    #print(time_Change)
    print("======================================")
    output_Data_Frame = pd.concat([output_Data_Frame.copy(),pd.DataFrame([[image_Name,len(centroids),deformable,nondeformable,other,time_Change]], columns = ["File Name", "Adhered Objects", "Deformable Counts", "Non-Deformable Counts", "Other", "Time (s)"])])
    display(output_Data_Frame)
print("Final Data")
display(output_Data_Frame)
output_Data_Frame.to_csv("Analysis_Output.csv")

Analyzing Movie_08.avi-76


<ipython-input-4-0ab4eef9ff1a>:150: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output_Data_Frame = pd.concat([output_Data_Frame.copy(),pd.DataFrame([[image_Name,len(centroids),deformable,nondeformable,other,time_Change]], columns = ["File Name", "Adhered Objects", "Deformable Counts", "Non-Deformable Counts", "Other", "Time (s)"])])


,File Name,Adhered Objects,Deformable Counts,Non-Deformable Counts,Other,Time (s)
0,Movie_08.avi-76.jpg,56,52,1,1,78.551824


Analyzing Movie_08.avi-106


,File Name,Adhered Objects,Deformable Counts,Non-Deformable Counts,Other,Time (s)
0,Movie_08.avi-76.jpg,56,52,1,1,78.551824
0,Movie_08.avi-106.jpg,56,53,0,0,74.846571


...

,File Name,Adhered Objects,Deformable Counts,Non-Deformable Counts,Other,Time (s)
0,Movie_08.avi-76.jpg,56,52,1,1,78.551824
0,Movie_08.avi-106.jpg,56,53,0,0,74.846571
0,Movie_08.avi-61.jpg,56,53,1,1,75.371431
0,Movie_08.avi-46.jpg,56,53,2,1,73.025929


Analyzing Movie_08.avi-91


,File Name,Adhered Objects,Deformable Counts,Non-Deformable Counts,Other,Time (s)
0,Movie_08.avi-76.jpg,56,52,1,1,78.551824
0,Movie_08.avi-106.jpg,56,53,0,0,74.846571
0,Movie_08.avi-61.jpg,56,53,1,1,75.371431
0,Movie_08.avi-46.jpg,56,53,2,1,73.025929
0,Movie_08.avi-91.jpg,57,53,2,1,75.350414


Analyzing Movie_08.avi-1


KeyboardInterrupt: 